In [56]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from difflib import SequenceMatcher as SM
import pandas as pd
import numpy as np
import time
import Levenshtein
import re
from newsapi import NewsApiClient


In [57]:
fuzz.ratio("this is a test", "this is a test!")
fuzz.token_sort_ratio("Amazon", "aMZon")

!pip install newsapi-python

You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [58]:
data = pd.read_csv('dataXData.csv')
company = data['Business Name'].unique()
company = company.astype(str)
data.head()

,Unnamed: 0,Business Name,Industry,City,State
0,0,3M Company,Industrials,NaN,USA
1,1,A.O. Smith Corp,Industrials,NaN,USA
2,2,Abbott Laboratories,Health Care,NaN,USA
3,3,AbbVie Inc.,Health Care,NaN,USA
4,4,Accenture plc,Information Technology,NaN,USA


In [60]:
pd.unique(data[['City']].values.ravel())

array([nan, 'Quebec', 'BC', ..., 'TEMPERANCE', 'DELMONT', 'MONACA'],
      dtype=object)

In [61]:
print(len(company))

940427


In [62]:
start=time.time()
response='y'
while(response=='y'):
    a=[]
    text2=input("Which text do you want to search? ")
    text1 = re.sub('[^A-Za-z0-9]+', '', text2)
    company = data['Business Name'].unique()
    company = company.astype(str)
    for i in range(0,len(company),1):
        search=company[i].lower()
        a.append(fuzz.token_sort_ratio(text1,search))

    #Finding the company names with maximum risk score aka fuzzy ratio
    ratio=np.array(a)
    c=np.where(ratio==ratio.max())
    maximum=max(a)
    top_list = company[c]
    print("Risk Score for",text2,"is",maximum,"for the brands",top_list)
    
    #Decision Logic for Classification
    if maximum>=90:
        print("High risk")
    elif (maximum<90 and maximum>=60):
        print("Medium Risk")
    else:
        print("No Risk")

    #Recording time taken to search the database
    end=time.time()
    print("Time taken:",round(end-start,3),"seconds")

    df=data.loc[data['Business Name'].isin(top_list)][['Business Name','Industry','City','State']]
    df1=df.drop_duplicates(keep='first')
    print(df1)
    response=input("Do you want to search again? [y/n] ")
print("Thanks for searching with us!")  

Which text do you want to search? apple inc
Risk Score for apple inc is 94 for the brands ['Apple Inc.' 'APPLE INC.' 'APPLE INC' 'Apple Inc' 'APPLE, INC.'
 'APPLE  INC']
High risk
Time taken: 30.051 seconds
        Business Name                                           Industry  \
51         Apple Inc.                             Information Technology   
2809       Apple Inc.                             Computer Manufacturing   
24144      APPLE INC.                                       RETAIL STORE   
43145       APPLE INC                    Other Computer Related Services   
114846      Apple Inc                                   Computer Systems   
232237      Apple Inc                                       Retail Trade   
352361    APPLE, INC.                         Computer & software stores   
644127    APPLE, INC.  All other miscellaneous store retailers (inclu...   
785676     APPLE  INC               All Other General Merchandise Stores   
798189    APPLE, INC.            

In [63]:
unique_names = []
for d in top_list:
    lower = d.lower()
    if lower not in unique_names:
        unique_names.append(lower)
        
print(unique_names)

['apple inc.', 'apple inc', 'apple, inc.', 'apple  inc']


In [72]:
from newsapi import NewsApiClient

entity_column = []
url_column = []
snippet_column = []
# Init
newsapi = NewsApiClient(api_key='a28acbbe0b2a46428906f353d180c5c3')
all_articles = []
for entity in unique_names:
    all_articles=newsapi.get_everything(q=entity,
                                         language='en', # necessary to only maintain english? probably for now
                                         sort_by='relevancy',
                                         page=1)["articles"]

    for article in all_articles:
        url = article["url"]        
        source = requests.get(url)
        if source.status_code != 200:
            continue
        text = text_from_html(source.content).decode('utf-8')
        
        if (' ' not in entity):
            splitted = text.split()
            lower = [x.lower() for x in splitted]

            try:
                index = lower.index(entity);

            except ValueError:
                index = -1
            if(index != -1):
                if(index < 10):
                    article_snippet = " ".join(splitted[0:index+10])
                else:
                    article_snippet = " ".join(splitted[index-10:index+10])
                entity_column.append(entity)
                url_column.append(url)
                snippet_column.append(article_snippet)
        else:
            indices = [m.start() for m in re.finditer(entity.lower(), text.lower())]
            
            if not indices: 
                continue
            index = indices[0]
            if(index < 30):
                article_snippet = text[0:index+30]
            else:
                article_snippet = text[index-30:index+30]
            entity_column.append(entity)
            url_column.append(url)
            snippet_column.append(article_snippet)
                
columns = {"entity" : entity_column, "url": url_column, "snippet" : snippet_column}
df_result = pd.DataFrame.from_dict(columns)
print(df_result.to_string())
        
        


        entity                                                url                                            snippet
0   apple inc.  https://www.reuters.com/article/us-apple-produ...  Shannon Stapleton (Reuters) - Apple Inc said o...
1   apple inc.  https://www.fastcompany.com/90272180/supreme-c...  stices will hear arguments in Apple Inc. v. Pe...
2   apple inc.  https://www.reuters.com/article/us-global-mark...   A steep decline in shares of Apple Inc ( AAPL...
3   apple inc.  https://www.reuters.com/article/us-apple-iphon...   raft of profit warnings from Apple Inc ( AAPL...
4   apple inc.  https://gizmodo.com/should-the-supreme-court-k...  8 10:00am Filed to: Pepper v. Apple Inc Filed ...
5   apple inc.  https://gizmodo.com/supreme-court-appears-to-l...  r Apple. The case of Pepper v Apple Inc  has t...
6   apple inc.        https://www.entrepreneur.com/article/322750  0.23 percent. Tech bellwether Apple Inc. faile...
7   apple inc.  https://www.androidcentral.com/samsung-expecte..